Name : Meghaben Rathwa , ID : 202001128
Name : Isha Shah , ID : 202001163
Name : Bhavajna Kallakuri , ID : 202003046

In [1031]:
import random
from qiskit import QuantumCircuit, Aer, execute,assemble,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, plot_bloch_multivector,plot_bloch_vector
from numpy.random import randint
import numpy as np
import qiskit.quantum_info as qi
from qiskit.quantum_info import random_statevector, Statevector
from qiskit import QuantumCircuit, execute
from qiskit.providers.aer import QasmSimulator

In [1032]:
# function for applying error of bit flip to all qubits
def apply_bit_error(qc, qubits, p):
    for qubit in qubits:
        if random.random() < p:
            qc.x(qubit)

In [1033]:
# function for applying error of phase flip to all qubits
def apply_phase_error(qc,qubits,p):
    for qubit in qubits:
        if random.random() < p:
            qc.z(qubit)

In [1034]:
# function for applying error of hadamard gates
def apply_hadamard(qc,qubits):
    for qubit in qubits:
        qc.h(qubit)

In [1035]:
def shor_circuit():
    # Define the number of qubits
    num_qubits = 1

    # Generate a random statevector

    random_state = random_statevector(2 ** num_qubits) # a|0> + b|1>
    
    # ket 0 state
    zero = Statevector([1,0])
    
    # Create the circuit
    qr= QuantumRegister(17)
    cr= ClassicalRegister(2)
    qc = QuantumCircuit(qr,cr)
    qc.initialize(random_state.data,qr[0])
    qc.save_statevector(label='v1')
    qc.barrier()
    initial = random_state ^ zero ^ zero ^ zero ^ zero ^ zero ^ zero ^ zero ^ zero  # inital vector is tensor product of 9 qubits
    
    #Encoding 
    # first apply c-not gate as taking control bit as 0
    qc.cx(qr[0],qr[3])
    qc.cx(qr[0],qr[6])
    qc.save_statevector(label='v2')
    qc.barrier()
    
    # now apply hadamard gates |+++> \--->
    qc.h(qr[0])
    qc.h(qr[3])
    qc.h(qr[6])
    
    # now apply c-not gates by taking control bit to 0 ,3 and 6 and similar to bit-flip codes
    qc.cx(qr[0],qr[1])
    qc.cx(qr[0],qr[2])
    
    qc.cx(qr[3],qr[4])
    qc.cx(qr[3],qr[5])
    
    qc.cx(qr[6],qr[7])
    qc.cx(qr[6],qr[8])
    
    qc.save_statevector(label='v3')
    qc.barrier()
    
    # Apply error
    # qc.z(qr[0])
    error_probability = 0.1  # Set the error probability to 0.1
    qubits_to_apply_error = [0, 1, 2,3,4,5,6,7,8]  # Apply error to all three qubits
    apply_bit_error(qc, qubits_to_apply_error, error_probability)  # bit error
    apply_phase_error(qc,qubits_to_apply_error,error_probability) # phase error
    qc.save_statevector(label='v4')
    qc.barrier()
    
    # apply hadamard to each qubit
    # apply_hadamard(qc,qubits_to_apply_error)
    
    # Error Detection
    
    # detect and correct bit error
    qc.cx(qr[0],qr[9])
    qc.cx(qr[1],qr[9])
    qc.cx(qr[0],qr[10])
    qc.cx(qr[2],qr[10])
    # measure 9,10 error syndrome
    qc.measure([qr[9],qr[10]],[cr[0],cr[1]])
    # 00 then ignore
    qc.x(qr[1]).c_if(cr,1);  # Apply an X gate to qubit 1 if the measurement result is 1  01 
    qc.x(qr[2]).c_if(cr,2);  # Apply an X gate to qubit 2 if the measurement result is 2  10 
    qc.x(qr[0]).c_if(cr,3);  # Apply an X gate to qubit 0 if the measurement result is 3  11 
    
    qc.cx(qr[3],qr[11])
    qc.cx(qr[4],qr[11])
    qc.cx(qr[3],qr[12])
    qc.cx(qr[5],qr[12])
    
    # measure 11,12 error syndrome
    qc.measure([qr[11],qr[12]],[cr[0],cr[1]])
    # 00 then ignore
    qc.x(qr[4]).c_if(cr,1);  # Apply an X gate to qubit 4 if the measurement result is 1  01 
    qc.x(qr[5]).c_if(cr,2);  # Apply an X gate to qubit 5 if the measurement result is 2  10 
    qc.x(qr[3]).c_if(cr,3);  # Apply an X gate to qubit 3 if the measurement result is 3  11 
    
    qc.cx(qr[6],qr[13])
    qc.cx(qr[7],qr[13])
    qc.cx(qr[6],qr[14])
    qc.cx(qr[8],qr[14])
    
    # measure 13,14 error syndrome
    qc.measure([qr[13],qr[14]],[cr[0],cr[1]])
    # 00 then ignore
    qc.x(qr[7]).c_if(cr,1);  # Apply an X gate to qubit 7 if the measurement result is 1  01 
    qc.x(qr[8]).c_if(cr,2);  # Apply an X gate to qubit 8 if the measurement result is 2  10 
    qc.x(qr[6]).c_if(cr,3);  # Apply an X gate to qubit 6 if the measurement result is 3  11 
    
    #detect and correct phase error
    apply_cx = [0,1,2,3,4,5]
    for q in apply_cx:
        qc.cz(q,qr[15])
    apply_cx = [3,4,5,6,7,8]
    for q in apply_cx:
        qc.cz(q,qr[16])

    qc.measure([qr[15],qr[16]],[cr[0],cr[1]])
    # 00 ignore
    # 01 apply z gate on any one qubit of last 3 qubits
    qc.z(qr[6]).c_if(cr,1)
    # 10 apply z gate on any one qubits of first 3 qubits
    qc.z(qr[0]).c_if(cr,2)
    # 11 apply z gate on any one qubits of middle 3 qubits
    qc.z(qr[3]).c_if(cr,3)
    
    #apply hadamard to each qubit
    # apply_hadamard(qc,qubits_to_apply_error)
    
    # Decoding
    # now apply c-not gates by taking control bit to 0 ,3 and 6 and similar to bit-flip codes
    qc.cx(qr[0],qr[1])
    qc.cx(qr[0],qr[2])
    
    qc.cx(qr[3],qr[4])
    qc.cx(qr[3],qr[5])
    
    qc.cx(qr[6],qr[7])
    qc.cx(qr[6],qr[8])
    qc.save_statevector(label='v5')
    qc.barrier()
    # now apply hadamard gates
    qc.h(qr[0])
    qc.h(qr[3])
    qc.h(qr[6])
    qc.save_statevector(label='v6')
    qc.barrier()
    # apply c-not gate as taking control bit as 0
    qc.cx(qr[0],qr[3])
    qc.cx(qr[0],qr[6])
    qc.save_statevector(label='v7')
    qc.barrier()
    return qc

In [1036]:
qc = shor_circuit()
qc.draw()

┌─────────────────────────────────────────────────┐ v1  ░           »
 q731_0: ┤ Initialize(-0.11933-0.77932j,-0.15565-0.59514j) ├─░───░───■────■──»
         └─────────────────────────────────────────────────┘ ░   ░   │    │  »
 q731_1: ────────────────────────────────────────────────────░───░───┼────┼──»
                                                             ░   ░   │    │  »
 q731_2: ────────────────────────────────────────────────────░───░───┼────┼──»
                                                             ░   ░ ┌─┴─┐  │  »
 q731_3: ────────────────────────────────────────────────────░───░─┤ X ├──┼──»
                                                             ░   ░ └───┘  │  »
 q731_4: ────────────────────────────────────────────────────░───░────────┼──»
                                                             ░   ░        │  »
 q731_5: ────────────────────────────────────────────────────░───░────────┼──»
                                                             ░   ░      ┌─┴─┐»
 q731_6: ────────────────────────────────────────────────────░───░──────┤ X ├»
                                                             ░   ░      └───┘»
 q731_7: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
 q731_8: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
 q731_9: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_10: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_11: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_12: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_13: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_14: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_15: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
q731_16: ────────────────────────────────────────────────────░───░───────────»
                                                             ░   ░           »
  c98: 2/════════════════════════════════════════════════════════════════════»
                                                                             »
«          v2  ░ ┌───┐           v3  ░ ┌───┐ v4  ░                          »
« q731_0: ─░───░─┤ H ├──■────■───░───░─┤ Z ├─░───░───■───────────────────■──»
«          ░   ░ └───┘┌─┴─┐  │   ░   ░ └───┘ ░   ░   │                   │  »
« q731_1: ─░───░──────┤ X ├──┼───░───░───────░───░───┼──────────────■────┼──»
«          ░   ░      └───┘┌─┴─┐ ░   ░ ┌───┐ ░   ░   │              │    │  »
« q731_2: ─░───░───────────┤ X ├─░───░─┤ Z ├─░───░───┼──────────────┼────┼──»
«          ░   ░ ┌───┐     └───┘ ░   ░ └───┘ ░   ░   │              │    │  »
« q731_3: ─░───░─┤ H ├──■────■───░───░───────░───░───┼────■─────────┼────┼──»
«          ░   ░ └───┘┌─┴─┐  │   ░   ░       ░   ░   │    │         │    │  »
« q731_4: ─░───░──────┤ X ├──┼───░───░───────░───░───┼────┼─────────┼────┼──»
«          ░   ░      └───┘┌─┴─┐ ░   ░ ┌───┐ ░   ░   │    │         │    │  »
« q731_5: ─░───░───────────┤ X ├─░───░─┤ X ├─░───░───┼────┼─────────┼────┼──»
«          ░   ░ ┌───┐     └───┘ ░   ░ └───┘ ░   ░   │    │         │    │  »
« q731_6: ─░───░─┤ H ├──■────■───░───░───────░───░───┼────┼────■────┼───

In [1037]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1)
result = job.result()
# counts = result.get_counts(qc)
# print(counts)
sv1 = result.data(0)['v1']
sv2 = result.data(0)['v2']
sv3 = result.data(0)['v3']
sv4 = result.data(0)['v4']
sv5 = result.data(0)['v5']
sv6 = result.data(0)['v6']
sv7 = result.data(0)['v7']


In [1038]:
sv1.draw('latex', prefix='sv1:')

<IPython.core.display.Latex object>

In [1039]:
sv2.draw('latex', prefix='sv2:')

<IPython.core.display.Latex object>

In [1040]:
sv3.draw('latex', prefix='sv3:')

<IPython.core.display.Latex object>

In [1041]:
sv4.draw('latex', prefix='sv4:')

<IPython.core.display.Latex object>

In [1042]:
sv5.draw('latex', prefix='sv5:')

<IPython.core.display.Latex object>

In [1043]:
sv6.draw('latex', prefix='sv6:')

<IPython.core.display.Latex object>

In [1044]:
sv7.draw('latex', prefix='sv7:')

<IPython.core.display.Latex object>